# Quantifying local lipid environments

The local environment of a lipid refers to the lipid composition of the patch of membrane surrounding a reference lipid. The local lipid environment can tell us whether there is a preferential aggregation of specific lipid species within a membrane.

We will first look at the local environment of lipids in a DPPC, DOPC, and cholesterol membrane simulated by [Smith et al.](https://www.biorxiv.org/content/10.1101/2021.05.24.445501v3). We will later perform similar calculations for the neuronal plasma membrane studied by [Ingólfsson et al. (2017)](https://www.cell.com/biophysj/fulltext/S0006-3495(17)31132-3).


In [1]:
import pathlib
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import MDAnalysis as mda
from lipyphilic.lib.neighbours import Neighbours


## Load the topology and trajectory using MDAnalysis

In [2]:
u = mda.Universe("../datafiles/dppc-dopc-chol.tpr", "../datafiles/dppc-dopc-chol.xtc")

## Generate a neighbour matrix

Before we quantigy local lipid environments, we must first use **lipyphilic** to construct an adjacency matrix, $A$, that describes whether each pair of lipid molecules are neighbouring one another or not. The neighbour matrix is defined as follows:

- $A_{ij}=1$ if lipid $i$ is neighbouring lipid $j$
- $A_{ij}=0$ otherwise

To construct the neighbour matrix, we will use the class `lipyphilic.lib.neighbours.Neighbours`.

In [3]:
neighbours = Neighbours(
    universe=u,
    lipid_sel="name GL1 GL2 ROH",
    cutoff=12.0
)

The above selection (`lipid_sel=name GL1 GL2 ROH`) will select the glycerol backbone of the phospholipids and the hydroxyl headgroup of cholesterol.

A phospholipid will be considered to be neighbouring a cholesterol molecule if either its *GL1* or *GL2* bead is within *12* Å of the ROH bead of the cholesterol. For neighbouring lipids, the distances between their respective *GL1* and "GL2* beads are be considered; that is, if two phosphoslipids have any of their glycerol backbone beads within *12* Å of one another, they will be considered neighbours.


We then select which frames of the trajectory to analyse (`None` will use every frame) and select to display a progress bar (`verbose=True`):
  

In [ ]:
neighbours.run(
    start=None,
    stop=None,
    step=None,
    verbose=True
)


  0%|          | 0/51 [00:00<?, ?it/s]

## Accessing the results

The results are stored in the `neighbours.Neighbours` attribute as a NumPy of Compressed Sparse Row matrices.

There is one matrix for each frame used in the analysis:

In [ ]:
neighbours.neighbours.size

Let's take a look at the first frame:

In [ ]:
first_matrix = neighbours.neighbours[0]

In [ ]:
first_matrix

This is a scipy Compressed Sparse Row matrix. It has a shape of ($N_{\rm lipids}, N_{\rm lipids}$)


We can extract pairs of neighbouring lipids as follows:

In [ ]:
reference, neighbour = first_matrix.nonzero()

In [ ]:
print(reference)

In [ ]:
print(neighbour)

We see that at this first frame, the lipid with residue index 0 is neighbouring lipids 916, 2265, 3492, and so on.

## Calculating the local lipid environment

Once we have constructed our adjacency matrices, we can use the `count_neighbours` helper method to quantify the local environment of each lipid at each frame:

In [ ]:
counts = neighbours.count_neighbours()


`counts` is a Pandas DataFrame in which each row contains the following information:

- lipid residue name
- lipid residue index
- frame number
- number of cholesterol neighbours
- number of DOPC neighbours
- number of DPPC neighbours
- total number of neighbours

Let's take a look at the DataFrame:


In [ ]:
counts

We see that at the first frame, the first cholesterol molecule (residue index 0) has:
- 3 cholesterol neighbours
- 3 DOPC neighbours
- 4 DPPC neighbours
- 10 neighbours in total


## Store the reuslts
There are a couple of ways of storing the results from the analysis.

Well store the results in the following directory:

In [ ]:
results_directory = pathlib.Path(
    "../results/neighbours"
)

# Create the directory if it doesn't already exist
results_directory.resolve().mkdir(exist_ok=True, parents=True)


1. You can persist the `Neighbours` object using pickle

In [ ]:
# Location to store the results
filename = results_directory.joinpath("neighbours-dppc-dopc-chol.pkl")
print(filename)

In [ ]:
# store the object
with open(filename, 'wb') as f:
    pickle.dump(neighbours, f)
    

In [ ]:
# load for later usage
with open(filename, 'rb') as f:
    neighbours = pickle.load(f)


2. You store the pandas DataFrame of the local lipid environment

In [ ]:
# Location to store the results
filename = results_directory.joinpath("neighbour-counts-dppc-dopc-chol.h5")
print(filename)

In [ ]:
# store the counts DataFrame
counts.to_hdf(
    filename,
    key='df',
    mode='w'
)


In [ ]:
# load for later usage
counts = pd.read_hdf(filename)


# Lipid enrichment/depletion index

The `count_neighbours` method will, by default, return the number of neighbouring lipid species around each individual lipid.

However, a clearer picture of aggregation of certain lipid species can be gained by instead considering the [enrichment/depletion index](https://pubs.acs.org/doi/10.1021/ja507832e) of each lipid species. 

The lipid enrichment index of species $B$ around species $A$, $E_{AB}$, may be defined as:
$$
E_{AB} = N_{AB} / \langle N_B \rangle
$$

where $N_{AB}$ is the number of molecules of species $B$ around species $A$, and $\langle N_B \rangle$ is the mean number of species $B$ around any species.

To calculate the enrichment/depletion index of each species at each frame, as well as the raw neighbour counts, we can set the `return_enrichment` keyword to `True` when we call `count_neighbours`:

In [ ]:
counts, enrichment = neighbours.count_neighbours(return_enrichment=True)

This will return two Pandas DataFrames, one containing the neighbour counts and the other the enrichment/depletion index of each species at each frame. 

`enrichment` contains the following information for each row:

- lipid species
- frame number
- enrichment of cholesterol
- enrichment of DOPC
- enrichment of DPPC

Let's take a look at the DataFrame:


In [ ]:
enrichment

**Values:**
- **below 1 correspond to depletion**
- **above 1 correspond to enrichment**


We see that at the first frame, the local environment of cholesterol is:
- depleted of cholesterol and DOPC
- enriched in DPPC

At the final frame, the local lipid environment of DPPC is:
- enriched in cholesterol and DPPC
- slightly depleted of DOPC


## Plot the enrichment over time

As we know the enrichment at each frame, we can plot the time-evolution of the enrichment/depletion index to see whether particular species form aggregates over time.


### Plotting with Pandas

In [ ]:
# Get a list of the species in the membrane
unique_labels = counts.Label.unique()

In [ ]:
# iterate over each species
for ref in unique_labels:
    
    # get the enrichment of the current species
    species_fe = enrichment.loc[enrichment.Label == ref].copy()
    
    # For plotting, we need to know the times used in the analysis
    ps_to_microseconds = 1e-6
    species_fe["Time"] = species_fe["Frame"].values * neighbours._trajectory.dt * ps_to_microseconds
    
    # For the current species, plot the enrichment of each other species over time
    species_fe.plot.line(
        x=r"Time",
        y=[f"fe{l}" for l in unique_labels],
        xlabel=r"Time ($\rm \mu s$)",
        ylabel="Enrichment"
    )
    
    # Add label info
    plt.title(f"Reference: {ref}", loc="left", weight="bold")
    plt.legend(unique_labels, loc="lower left", bbox_to_anchor=(1.01, 0.0), frameon=False, title="Neighbour")
    
    plt.hlines(y=1.0, xmin=0, xmax=0.25, linestyle="--", color="xkcd:grey", alpha=0.8)
    plt.show()
    

The trajectory we are using is a 250 ns time period from 7.75 µs to 8.0 µs. By this point, cholesterol and DPPC have demixed somewhat from DOPC, whilst DOPC has self-aggregated.

# Counting the number of $L_d$ or $L_o$ neighbours

Instead of using the lipid resname to identify neighbouring lipids, any ordinal data may be used for counting lipid neighbours. This is done through use of the `count_by` and `count_by_labels` parameters.

[In another tutorial](HiddenMarkovModel.ipynb), we construct a hidden Markov model based on lipid thicknesses in order to determine whether each lipid in each frame is liquid-ordered ($L_o$) or liquid-disordered ($L_d$). We will use this information here in order to quantify to local lipid environments.


In [ ]:
# Load the lipid order data
filename = "../results/HMM/lipid-order.npy"
lipid_order = np.load(filename)


`lipid_order` is a two-dimensional NumPy. Like many analyses in **lipyphilic**, the array is of shape ($N_{\rm lipids}$, $N_{\rm lipids}$) The:

- rows correspond to individual lipids
- columns correspond to individual frames.

In the array, the ordered state of each lipid is defined as follows:

- values of 1 indicate the lipid is in the $Ld$ state
- values of -1 indicate the lipid is in the $Ld$ state

Let's look at the array:


In [ ]:
lipid_order

We see the first lipid (first row) is $L_o$ both at the beginning and end of the frames used in the analysis.

The second lipid (second row) is $L_o$ at the beginning of the analysis but $L_d$ at the end.



## Pass the lipid order data to `count_neighbours`

Because `lipid_order` has shape $N_{\rm lipids}$, $N_{\rm lipids}$, we can pass this NumPy as input to `count_neighbours`. `count_neighbours` will then use the lipid order data to define local lipid environments:


In [ ]:
counts = neighbours.count_neighbours(
    count_by=lipid_order,
    count_by_labels={'Ld': -1, 'Lo': 1}  # tell count_neighbours that -1 and 1 correspond to Ld and Lo respectively
)

As before, `counts` is a Pandas DataFrame.

However, this time, each row contains the following information:

- lipid order ($L_d$ or $L_o$)
- lipid residue index
- frame number
- number of $L_d$ neighbours
- number of $L_o$ neighbours

Let's take a look at the DataFrame:

    

In [ ]:
counts

At the first frame, the lipid with residue index 0 is in the $L_o$ state. It has 3 $L_d$, neighbours, 7 $L_o$ neighbours, and 10 neighbours in total.

## Enrichment/depletion of $L_d$ or $L_o$ lipids

As before, we can set `return_enrichment=True` to obtain the enrichment at each frame:

In [ ]:
counts, enrichment = neighbours.count_neighbours(
    count_by=lipid_order,
    count_by_labels={'Ld': -1, 'Lo': 1},  # tell count_neighbours that -1 and 1 correspond to Ld and Lo respectively
    return_enrichment=True
)

`enrichment` is a Pandas Dataframe that contains the enrichment/depletion index of each ordered state.

`enrichment` contains the following information for each row:

- lipid order ($L_d$ or $L_o$)
- frame number
- enrichment of $L_d$
- depletion of $L_o$


In [ ]:
enrichment

# More complex mixtures

The construction of the neighbour matrix, and quantification of local lipid environments, makes use of the powerful selection language of MDAnalysis. This means local lipid environments can readily be calculated even for membranes with highly complex compositions.

Below we will quantigy to local lipid environment for each species in the 58-component neuronal plasma membrane studied by [Ingólfsson et al. (2017)](https://www.cell.com/biophysj/fulltext/S0006-3495(17)31132-3).


In [ ]:
u = mda.Universe("../datafiles/LIM25_neuronal_plasma_membrane.gro")

In [ ]:
membrane = u.select_atoms("name GL1 GL2 AM1 AM2 ROH").residues

The above selects all lipids in our system:

In [ ]:
print(f"Number of lipids: {membrane.n_residues}")
print(f"Number of lipid species: {np.unique(membrane.resnames).size}")

## Generate the neighbour matrix


In [ ]:
# lipid_sel cover all 58 species in the membrane
neighbours = Neighbours(
    universe=u,
    lipid_sel="name GL1 GL2 AM1 AM2 ROH",
    cutoff=12.0
)


In [ ]:
# Construct the neighbour matrix
neighbours.run(
    start=None,
    stop=None,
    step=None,
    verbose=True
)


## Get the neighbour counts and enrichment/depletion of each lipid species

In [ ]:
counts, enrichment = neighbours.count_neighbours(
    return_enrichment=True
)

`counts` contains the number of each species around each individual lipid at each frame.

In [ ]:
counts.head()

`enrichment` containts the enrichment/depletion index for each species at each frame

In [ ]:
enrichment.head()

## Plot a heatmap of the lipid enrichment/depletion

In [ ]:
# Average the enricment values over time
# Extract the enrichment value into a two-dimensional NumPy array
enrichment_values = enrichment.groupby("Label")[enrichment.columns[2:]].mean().values

In [ ]:
# There is an enrichment value for each species around each other species
enrichment_values.shape

In [ ]:
# Extract the species residue names to label our heatmap
labels = np.asarray([column.strip("fe") for column in enrichment.columns if column.startswith("fe")])

In [ ]:
# This is a NumPy array containing each species name in the same order as seen in the enrichment DataFrame
labels

Now we can create our heatmap:

In [ ]:
# Create a huge figure large enough for our 58 x 58 matrix
plt.figure(figsize=(50, 50))
sns.set({"font.size": 15})

sns.heatmap(
    enrichment_values,
    xticklabels=labels,
    yticklabels=labels,
    vmin=0, vmax=3, center=1,
    annot=False, fmt=".1f",
    square=True,
    cmap="RdBu",
    cbar=True,
    cbar_kws={"label": "Enrichment", "aspect":100, "shrink": 0.863, "pad": 0.01}
)

plt.xlabel("Neighbour")
plt.ylabel("Reference")


## Plotting enrichment by tail saturation
The above heatmap, which plots the enrichment of each species around every other species, contains too much information to be of much use.

Here, we will instead calculate the enrichment of lipids based on how saturated their tails are.


In [ ]:
lipids = np.asarray([column.strip("fe") for column in enrichment.columns if column.startswith("fe")])

First we need to define the saturation of each lipid:

In [ ]:
membrane = u.select_atoms("name GL1 GL2 AM1 AM2 ROH")
saturation = []

# Determine the saturation of each individual lipid
for lipid in np.unique(membrane.resnames):
    
    # select only the lipids of the current species
    lipid_residues = membrane.residues.atoms.select_atoms(f"resname {lipid}").residues
    
    # now we will consider a single lipid of this species to determine its saturation
    lipid_atoms = lipid_residues[0].atoms
    
    # It is a sterol if it has an ROH bead
    if "ROH" in lipid_atoms.names:
        saturation.append("Chol")
    
    else:
        
        # count the number of D beads - these refer to double bonds in the acyl tails
        num_doulbe = sum([True for name in lipid_atoms.names if name.startswith("D")])
        
        # It is saturated if it has no D beads
        if num_doulbe == 0:
            saturation.append("S")
            
        # It is monounsaturated if it has 1 D bead
        elif num_doulbe == 1:
            saturation.append("MU")
            
        # It is polyunsaturated if it has 2+ D beads
        else:
            saturation.append("PU")

saturation = np.array(saturation)


Now we will create a two-dimensional NumPy array, of shape ($N_{\rm lipids}, N_{\rm frames}$), where each lipid is labelled by its saturation.


In [ ]:
count_by = np.full((membrane.n_residues, neighbours.n_frames), fill_value="", dtype="<U4")

for lipid, sat in zip(np.unique(membrane.resnames), saturation):
    
    lipid_indices = membrane.residues.resnames == lipid
    count_by[lipid_indices] = sat


We can now pass this `count_by` array to `count_neighbours`

In [ ]:
counts, enrichment = neighbours.count_neighbours(
    count_by=count_by,
    return_enrichment=True
)

Let's looks at the DataFrames:

In [ ]:
counts.head()

In [ ]:
enrichment.head()

Now we can plot enrichment in terms of tail saturation

In [ ]:
# Average the enricment values over time
# Extract the enrichment value into a two-dimensional NumPy array
enrichment_values = enrichment.groupby("Label")[enrichment.columns[2:]].mean().values

In [ ]:
# This is now a NumPy that contains the enrichment/depletion in terms of tail saturation
enrichment_values

In [ ]:
ticklabels = ["Chol", "Sat", "Mono Unsat", "Poly Unsat"]
plt.figure(figsize=(8, 8))
sns.set({"font.size": 15})

sns.heatmap(
    enrichment_values,
    vmin=0.5, vmax=1.5, center=1,
    annot=True, fmt=".3f",
    square=True,
    cmap="RdBu",
    cbar_kws={"label": "Enrichment", "aspect":40, "shrink": 0.84, "pad": 0.025}
)

plt.xlabel("Neighbour")
plt.ylabel("Reference")

plt.gca().set_xticklabels(ticklabels, rotation=45)
plt.gca().set_yticklabels(ticklabels, rotation=45)

plt.show()
